In [58]:
!pip install navec slovnet pymystem3 spacy gdown

In [5]:
!wget https://storage.yandexcloud.net/natasha-navec/packs/navec_news_v1_1B_250K_300d_100q.tar
!wget https://storage.yandexcloud.net/natasha-slovnet/packs/slovnet_ner_news_v1.tar

--2023-11-21 09:55:03--  https://storage.yandexcloud.net/natasha-navec/packs/navec_news_v1_1B_250K_300d_100q.tar
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26634240 (25M) [application/x-tar]
Saving to: ‘navec_news_v1_1B_250K_300d_100q.tar.1’

navec_news_v1_1B_25 100%[===================>]  25.40M  8.71MB/s    in 2.9s    

2023-11-21 09:55:07 (8.71 MB/s) - ‘navec_news_v1_1B_250K_300d_100q.tar.1’ saved [26634240/26634240]

--2023-11-21 09:55:08--  https://storage.yandexcloud.net/natasha-slovnet/packs/slovnet_ner_news_v1.tar
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2385920 (2.3M) [app

In [61]:
!gdown --id 1DNilv-qiNFjCSOhBFZE4JjGbSrAYZ9UP --output "all brands - Sheet2.csv"

/opt/conda/lib/python3.10/site-packages/gdown/cli.py:126: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1DNilv-qiNFjCSOhBFZE4JjGbSrAYZ9UP
To: /kaggle/working/all brands - Sheet2.csv
100%|██████████████████████████████████████| 12.2k/12.2k [00:00<00:00, 40.8MB/s]


In [6]:
import pickle
import pandas as pd
import numpy as np
import re

from navec import Navec
from slovnet import NER
from razdel import sentenize

from pymystem3 import Mystem
import spacy
import nltk

from tqdm import tqdm

In [7]:
import os
os.listdir('/kaggle/input/')

['sentiment', 'brands-without-rx', 'allbrands']

In [8]:
with open('../input/sentiment/messages_many.pkl', 'rb') as file:

    mes_many = pickle.load(file)

In [9]:
mentions = pd.read_csv('../input/sentiment/mentions.csv', index_col = 0)
mentions = mentions[mentions['issuerid'] > 0]
mentions = mentions.rename(columns={'messageid': 'MessageID'})

In [10]:
sentiment = pd.read_csv('../input/sentiment/sentiment.csv', index_col = 0)
sentiment = sentiment[sentiment['issuerid'] > 0]

In [11]:
sentiment

,MessageID,ChannelID,issuerid,SentimentScore
74,556,1155666448,150,3
75,559,1155666448,129,3
76,561,1155666448,99,4
79,562,1155666448,99,4
80,565,1155666448,150,3
...,...,...,...,...
4374,237595,1203560567,26,4
4375,237600,1203560567,53,4
4376,237650,1203560567,53,3
4377,237675,1203560567,129,4


In [12]:
mes_many

,MessageID,ChannelID,DateAdded,DatePosted,MessageText,IsForward
0,1,1001029560,2021-02-06 00:52:00,2015-11-18 19:09:32,,False
1,2,1001029560,2021-02-06 00:52:00,2015-11-23 13:02:29,http://bcs-express.ru/investidei/usdrubukoil/n...,False
2,3,1001029560,2021-02-06 00:52:00,2015-11-23 13:04:00,http://bcs-express.ru/novosti-i-analitika/inte...,False
3,4,1001029560,2021-02-06 00:52:00,2015-11-23 13:04:16,http://bcs-express.ru/novosti-i-analitika/priz...,False
4,5,1001029560,2021-02-06 00:52:00,2015-11-23 13:04:41,http://bcs-express.ru/novosti-i-analitika/kak-...,False
...,...,...,...,...,...,...
594604,1435,1565800335,2023-04-14 18:45:36,2023-04-14 07:30:00,"""​🖥 Softline / Noventiq Holdings (SFTL) - что...",False
594605,1437,1565800335,2023-04-17 14:25:45,2023-04-15 07:30:00,"​🔬 IPO ЦГМК Genetico, разбор компании ▫️Капит...",False
594606,1439,1565800335,2023-04-17 14:25:45,2023-04-16 19:00:02,"""Как купить участок за 3% от кадастровой стоим...",False
594607,1440,1565800335,2023-04-17 14:25:45,2023-04-17 10:15:36,⚡ Актуальные идеи на российском рынке Практич...,False


In [13]:
mes_0 = pd.read_excel('../input/sentiment/messages_0.xlsx', sheet_name=None)
for sheet_name in mes_0.keys():
    mes_0[sheet_name]['ticker'] = sheet_name
mes_0 = pd.concat(mes_0, ignore_index=True)

/tmp/ipykernel_47/910650669.py:4: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  mes_0 = pd.concat(mes_0, ignore_index=True)


In [14]:
mes_1 = pd.read_excel('../input/sentiment/messages_1.xlsx', sheet_name=None)
for sheet_name in mes_1.keys():
    mes_1[sheet_name]['ticker'] = sheet_name
mes_1 = pd.concat(mes_1, ignore_index=True)

/tmp/ipykernel_47/1859392702.py:4: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  mes_1 = pd.concat(mes_1, ignore_index=True)


In [15]:
mes_2 = pd.read_excel('../input/sentiment/messages_2.xlsx', sheet_name=None)
for sheet_name in mes_2.keys():
    mes_2[sheet_name]['ticker'] = sheet_name
mes_2 = pd.concat(mes_2, ignore_index=True)

/tmp/ipykernel_47/735663529.py:4: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  mes_2 = pd.concat(mes_2, ignore_index=True)


In [16]:
mes_012 = pd.concat([mes_0, mes_1, mes_2], ignore_index=True)

In [17]:
mes_012

,MessageText,DatePosted,MyScore,MessageID,ChannelID,ChannelURL,MessageLink,ticker
0,❗️🛢🇷🇺#газ #россия #макро Новак: запасы газа н...,2021-12-29 08:51:59,NaN,172344,1203560567,https://t.me/markettwits,https://t.me/markettwits/172344,nvtk
1,ПАО «НОВАТЭК» Тикер NVTK Рекомендация - покупк...,2021-12-23 13:38:22,NaN,13848,1300011741,https://t.me/finamalert,https://t.me/finamalert/13848,nvtk
2,🟢 НОВАТЭК договорился о поставках низкоуглерод...,2021-12-23 13:31:19,NaN,2473,1239405989,https://t.me/SberInvestments,https://t.me/SberInvestments/2473,nvtk
3,☀️ 23.12.2021 📰 Главные темы: • Отбор газа из...,2021-12-23 09:30:01,NaN,10636,1075101206,https://t.me/AK47pfl,https://t.me/AK47pfl/10636,nvtk
4,🇷🇺🇪🇺#NVTK #водород #аэ #европа НОВАТЭК поста...,2021-12-23 07:47:03,NaN,171722,1203560567,https://t.me/markettwits,https://t.me/markettwits/171722,nvtk
...,...,...,...,...,...,...,...,...
7533,🔦 Фокус недели #ФН Сегодня 🖨 Мосбиржа (MOEX)...,2021-12-02 08:01:34,NaN,29123,1197210433,https://t.me/cbrstocks,https://t.me/cbrstocks/29123,afks
7534,🗓КАЛЕНДАРЬ НА СЕГОДНЯ - 2021.12.02 🔥ГОРЯЧЕЕ 🛢...,2021-12-02 06:30:24,NaN,168528,1203560567,https://t.me/markettwits,https://t.me/markettwits/168528,afks
7535,"""✅ Главное по секторам российского рынка ☢️ У...",2021-12-01 19:00:00,NaN,10455,1075101206,https://t.me/AK47pfl,https://t.me/AK47pfl/10455,afks
7536,🇷🇺#AFKS #отчетность #прогноз Газпромбанк: АФК,2021-12-01 10:08:19,NaN,168347,1203560567,https://t.me/markettwits,https://t.me/markettwits/168347,afks


In [18]:
text_ = 'ЕПАО «НОВАТЭК» Тикер NVTK Рекомендация - покупка ⬆️ Инвестиционный горизонт 1-2 месяца Цена входа - тек. рыночная Цель 2 080 руб. Потенциальная доходность на сделку 22% Объем входа 5% Стоп-приказ 1 500 руб.   Технический анализ Бумага оттолкнулась от уровня поддержки. Открывать позиции рекомендуем от текущих уровней с целью 2 080 рублей. При покупке на 5% от объема портфеля и выставлении стоп-заявки на уровне 1500 рублей риск на портфель составит 0,60%. Соотношение прибыль/риск составляет 1,9.   Фундаментальный фактор Основным позитивным моментом в инвестиционном кейсе «НОВАТЭКа» является его программа развития в сфере СПГ. К 2030 году компания планирует увеличить производство СПГ в 2,9–3,6 раза до 57–70 млн тонн. Благодаря стабильно растущему спросу, нет сомнений, что «НОВАТЭК» найдет покупателей на свою продукцию. При этом значительные налоговые льготы и низкие расходы на добычу и сжижение газа делают российский СПГ одним из самых конкурентоспособных в мире. В текущем году «НОВАТЭК», как и все нефтегазовые компании, показывает восстановление финансовых результатов. Причинами для этого стали рост цен на нефть и СПГ, развитие проекта «Ямал СПГ», а также небольшое увеличение объемов добычи углеводородов. На этом фоне мы полагаем, что в 2021 году все основные финансовые показатели «НОВАТЭКа» будут рекордными, как и в новом 2022 году.'

navec = Navec.load('navec_news_v1_1B_250K_300d_100q.tar')
ner = NER.load('slovnet_ner_news_v1.tar')
ner.navec(navec)
markup_ = ner(text_)
print(markup_.text[15:38])
print(text_[15:38])

Тикер NVTK Рекомендация
Тикер NVTK Рекомендация


In [19]:
mes_cp = mes_012[:10]
mes_cp = mes_cp[['MessageText', 'DatePosted','ticker']]

In [20]:
%%timeit
mes_cp = mes_012[:10]
mes_cp = mes_cp[['MessageText', 'DatePosted','ticker']]
for i in range(mes_cp.shape[0]):
    markup = ner(mes_cp.MessageText[i])
    string = set()
    rows_list = []
    MT = []
    cur_start = 0
    for span in markup.spans:  
        if span.type == 'ORG':
            word = (markup.text[span.start:span.stop]).lower()
            if word not in string: # долгая проверка
                string.add(word)
                MT.append(mes_cp.MessageText[i][cur_start:span.stop])
                cur_start = span.stop
    DP = mes_cp.DatePosted[i]
    for s in string:
        c = 0
        new_data = {'MessageText': MT[c], 'DatePosted': DP, 'ticker':s}
        rows_list.append(new_data)
        c+=1
    df = pd.DataFrame(rows_list)
    mes_cp = pd.concat([mes_cp, df], ignore_index=True)  

187 ms ± 3.02 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [21]:
mes_cp

,MessageText,DatePosted,ticker
0,❗️🛢🇷🇺#газ #россия #макро Новак: запасы газа н...,2021-12-29 08:51:59,nvtk
1,ПАО «НОВАТЭК» Тикер NVTK Рекомендация - покупк...,2021-12-23 13:38:22,nvtk
2,🟢 НОВАТЭК договорился о поставках низкоуглерод...,2021-12-23 13:31:19,nvtk
3,☀️ 23.12.2021 📰 Главные темы: • Отбор газа из...,2021-12-23 09:30:01,nvtk
4,🇷🇺🇪🇺#NVTK #водород #аэ #европа НОВАТЭК поста...,2021-12-23 07:47:03,nvtk
5,✅ Главное по секторам российского рынка ☢️ Ур...,2021-12-22 19:29:59,nvtk
6,💥🇷🇺🇩🇪#NVTK #германия #аммиак Новатэк и немецка...,2021-12-22 13:35:02,nvtk
7,#NVTK ⛽️ Новатэк (NVTK) и Uniper подписали сог...,2021-12-22 13:34:39,nvtk
8,🟢 НОВАТЭК утвердил программу выкупа акций на $...,2021-12-20 15:37:05,nvtk
9,"🇷🇺#NVTK #buyback Новатэк, в период с 13 по 17...",2021-12-20 15:22:15,nvtk


## #todo lemma + word in issuers

In [22]:
# lemma только для русских слов тк это сущетсвительное ✅
# потом уже делать in issuers ✅ ~
# объеденить issuers компании в строку и делать word substring
# делить по предложениям а не по spans index ✅

In [23]:
text = "Красивая SberCIB"
m = Mystem() #pymystem3
lemmas = m.lemmatize(text)
lemmas

Installing mystem to /root/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-linux-64bit.tar.gz


['красивый', ' ', 'SberCIB', '\n']

In [24]:
#%%timeit
mes_cp = mes_012[:20]
mes_cp = mes_cp[['MessageText', 'DatePosted','ticker']]
for i in range(mes_cp.shape[0]):
    markup = ner(mes_cp.MessageText[i])
    string = set()
    rows_list = []
    MT = []
    cur_start = 0
    for span in markup.spans:  
        if span.type == 'ORG':
            word = (markup.text[span.start:span.stop]).lower()
            word = m.lemmatize(word)[0]
            if word not in string: # долгая проверка
                string.add(word)
                MT.append(mes_cp.MessageText[i][cur_start:span.stop])
                cur_start = span.stop
    DP = mes_cp.DatePosted[i]
    for s in string:
        c = 0
        new_data = {'MessageText': MT[c], 'DatePosted': DP, 'ticker':s}
        rows_list.append(new_data)
        c+=1
    df = pd.DataFrame(rows_list)
    mes_cp = pd.concat([mes_cp, df], ignore_index=True)  

In [25]:
mes_cp

,MessageText,DatePosted,ticker
0,❗️🛢🇷🇺#газ #россия #макро Новак: запасы газа н...,2021-12-29 08:51:59,nvtk
1,ПАО «НОВАТЭК» Тикер NVTK Рекомендация - покупк...,2021-12-23 13:38:22,nvtk
2,🟢 НОВАТЭК договорился о поставках низкоуглерод...,2021-12-23 13:31:19,nvtk
3,☀️ 23.12.2021 📰 Главные темы: • Отбор газа из...,2021-12-23 09:30:01,nvtk
4,🇷🇺🇪🇺#NVTK #водород #аэ #европа НОВАТЭК поста...,2021-12-23 07:47:03,nvtk
5,✅ Главное по секторам российского рынка ☢️ Ур...,2021-12-22 19:29:59,nvtk
6,💥🇷🇺🇩🇪#NVTK #германия #аммиак Новатэк и немецка...,2021-12-22 13:35:02,nvtk
7,#NVTK ⛽️ Новатэк (NVTK) и Uniper подписали сог...,2021-12-22 13:34:39,nvtk
8,🟢 НОВАТЭК утвердил программу выкупа акций на $...,2021-12-20 15:37:05,nvtk
9,"🇷🇺#NVTK #buyback Новатэк, в период с 13 по 17...",2021-12-20 15:22:15,nvtk


### Sentence tokenizer (razdel/nltk)

In [26]:
text__ = 'ПАО «НОВАТЭК» Тикер NVTK Рекомендация - покупка ⬆️ Инвестиционный горизонт 1-2 месяца Цена входа - тек. рыночная Цель 2 080 руб. Потенциальная доходность на сделку 22% Объем входа 5% Стоп-приказ 1 500 руб.   Технический анализ Бумага оттолкнулась от уровня поддержки. Открывать позиции рекомендуем от текущих уровней с целью 2 080 рублей. При покупке на 5% от объема портфеля и выставлении стоп-заявки на уровне 1500 рублей риск на портфель составит 0,60%. Соотношение прибыль/риск составляет 1,9.   Фундаментальный фактор Основным позитивным моментом в инвестиционном кейсе «НОВАТЭКа» является его программа развития в сфере СПГ. К 2030 году компания планирует увеличить производство СПГ в 2,9–3,6 раза до 57–70 млн тонн. Благодаря стабильно растущему спросу, нет сомнений, что «НОВАТЭК» найдет покупателей на свою продукцию. При этом значительные налоговые льготы и низкие расходы на добычу и сжижение газа делают российский СПГ одним из самых конкурентоспособных в мире. В текущем году «НОВАТЭК», как и все нефтегазовые компании, показывает восстановление финансовых результатов. Причинами для этого стали рост цен на нефть и СПГ, развитие проекта «Ямал СПГ», а также небольшое увеличение объемов добычи углеводородов. На этом фоне мы полагаем, что в 2021 году все основные финансовые показатели «НОВАТЭКа» будут рекордными, как и в новом 2022 году.  ✅ Пишите, есл'

mmm = list(sentenize(text__))
mmm



[Substring(0,
           128,
           'ПАО «НОВАТЭК» Тикер NVTK Рекомендация - покупка ⬆️ Инвестиционный горизонт 1-2 месяца Цена входа - тек. рыночная Цель 2 080 руб.'),
 Substring(129,
           205,
           'Потенциальная доходность на сделку 22% Объем входа 5% Стоп-приказ 1 500 руб.'),
 Substring(208,
           267,
           'Технический анализ Бумага оттолкнулась от уровня поддержки.'),
 Substring(268,
           338,
           'Открывать позиции рекомендуем от текущих уровней с целью 2 080 рублей.'),
 Substring(339,
           456,
           'При покупке на 5% от объема портфеля и выставлении стоп-заявки на уровне 1500 рублей риск на портфель составит 0,60%.'),
 Substring(457, 497, 'Соотношение прибыль/риск составляет 1,9.'),
 Substring(500,
           630,
           'Фундаментальный фактор Основным позитивным моментом в инвестиционном кейсе «НОВАТЭКа» является его программа развития в сфере СПГ.'),
 Substring(631,
           722,
           'К 2030 году компания пла

In [27]:
sent_tokenizer = nltk.tokenize.PunktSentenceTokenizer()
text = 'ПАО «НОВАТЭК» Тикер NVTK Рекомендация - покупка ⬆️ Инвестиционный горизонт 1-2 месяца Цена входа - тек. рыночная Цель 2 080 руб. Потенциальная доходность на сделку 22% Объем входа 5% Стоп-приказ 1 500 руб.   Технический анализ Бумага оттолкнулась от уровня поддержки. Открывать позиции рекомендуем от текущих уровней с целью 2 080 рублей. При покупке на 5% от объема портфеля и выставлении стоп-заявки на уровне 1500 рублей риск на портфель составит 0,60%. Соотношение прибыль/риск составляет 1,9.   Фундаментальный фактор Основным позитивным моментом в инвестиционном кейсе «НОВАТЭКа» является его программа развития в сфере СПГ. К 2030 году компания планирует увеличить производство СПГ в 2,9–3,6 раза до 57–70 млн тонн. Благодаря стабильно растущему спросу, нет сомнений, что «НОВАТЭК» найдет покупателей на свою продукцию. При этом значительные налоговые льготы и низкие расходы на добычу и сжижение газа делают российский СПГ одним из самых конкурентоспособных в мире. В текущем году «НОВАТЭК», как и все нефтегазовые компании, показывает восстановление финансовых результатов. Причинами для этого стали рост цен на нефть и СПГ, развитие проекта «Ямал СПГ», а также небольшое увеличение объемов добычи углеводородов. На этом фоне мы полагаем, что в 2021 году все основные финансовые показатели «НОВАТЭКа» будут рекордными, как и в новом 2022 году.  ✅ Пишите, есл'
#m = sent_tokenizer.tokenize(text)


In [28]:
#%%timeit
m = Mystem()
mes_cp = mes_012[:1000]
mes_cp = mes_cp[['MessageText', 'DatePosted','ticker']]

for i in tqdm(range(mes_cp.shape[0])):
    sentences = list(sentenize(mes_cp.MessageText[i]))
    markup = ner(mes_cp.MessageText[i])
    string = set()
    
    MT = []
    for span in markup.spans:  
        if span.type == 'ORG':
            word = (markup.text[span.start:span.stop]) #.lower()
            word = m.lemmatize(word)[0]
            if word not in string: # долгая проверка
                string.add(word)
                ### text boundaries
                for s in sentences:
                    rr = []
                    for r in s: # (rangeL:rangeR), sentence
                        rr.append(r)    
                    if (span.start >= rr[0] and span.stop <= rr[1]):
                        MT.append(rr[2])   
                    
    rows_list = []            
    DP = mes_cp.DatePosted[i]
    for s in string:
        c = 0
        new_data = {'MessageText': MT[c], 'DatePosted': DP, 'ticker':s}
        rows_list.append(new_data)
        c+=1
    df = pd.DataFrame(rows_list)
    mes_cp = pd.concat([mes_cp, df], ignore_index=True)  


100%|██████████| 1000/1000 [00:26<00:00, 38.44it/s]


In [29]:
mes_cp

,MessageText,DatePosted,ticker
0,❗️🛢🇷🇺#газ #россия #макро Новак: запасы газа н...,2021-12-29 08:51:59,nvtk
1,ПАО «НОВАТЭК» Тикер NVTK Рекомендация - покупк...,2021-12-23 13:38:22,nvtk
2,🟢 НОВАТЭК договорился о поставках низкоуглерод...,2021-12-23 13:31:19,nvtk
3,☀️ 23.12.2021 📰 Главные темы: • Отбор газа из...,2021-12-23 09:30:01,nvtk
4,🇷🇺🇪🇺#NVTK #водород #аэ #европа НОВАТЭК поста...,2021-12-23 07:47:03,nvtk
...,...,...,...
4991,Канада выступает за отключение РФ от SWIFT Е...,2022-02-26 09:04:50,цб
4992,Канада выступает за отключение РФ от SWIFT Е...,2022-02-26 09:04:50,севмаш
4993,Канада выступает за отключение РФ от SWIFT Е...,2022-02-26 09:04:50,свр
4994,Канада выступает за отключение РФ от SWIFT Е...,2022-02-26 09:04:50,совкомфлот


### brands - все issuers(кроме полных названий) с повторами.
### Идея прогнать алгоритм выше на всех 7.5к строках. получить столбец с ticker'ами. Для каждого тикера проверить not in brands: если true, то поместить его в stopwords.
### потом апргрейднуть алгоритм проверяя уже not in stopwords.

In [30]:
brands = pd.read_csv('../input/allbrands/all brands.csv') 
brands['abraudurso'] = brands['abraudurso'].str.lower()
brand_list = brands.abraudurso.tolist()
brand_list

['abrd',
 'acron',
 'aeroflot',
 'afks rx',
 'aflt rx',
 'agro rx',
 'akrn rx',
 'alnu rx',
 'alrosa',
 'alrosa',
 'alrs rx',
 'amez',
 'ao fgc ues',
 'aptk rx',
 'aqua',
 'arsa',
 'arsagera(не встречал упоминаний)',
 'assb',
 'avan',
 'avangard',
 'bane rx',
 'belon',
 'belu rx',
 'beluga',
 'bisv                ',
 'blng',
 'bspb rx',
 'cbom rx',
 'central telegraph',
 'chelyabinsk metallurgical plant',
 'chelyabinsk pipe rolling plant',
 'chep rx',
 'cherkizovo',
 'chmf rx',
 'chmk rx',
 'ckpz rx',
 'cntl rx',
 'credit bank',
 'credit bank of moscow',
 'dagestanskaya energosbytovaya',
 'dasb',
 'derzp',
 'detsky mir',
 'diod plant of ecological technics and ecomeals',
 'diod rx',
 'dsky rx',
 'dvec',
 'dzrd',
 'eelt                ',
 'electrozink',
 'eltz rx                ',
 'en+ group',
 'en+ group plc',
 'enel russia',
 'enpg rx',
 'enpl',
 'enru rx',
 'etln li',
 'fees rx',
 'fesco',
 'fesh                ',
 'five li',
 'fixp rx',
 'flot rx',
 'gas-service',
 'gaz-tec',
 'gaz

In [31]:
#%%timeit
m = Mystem()
mes_cp_full = mes_012
mes_cp_full = mes_cp_full[['MessageText', 'DatePosted','ticker']]

for i in tqdm(range(mes_cp_full.shape[0])):
    sentences = list(sentenize(mes_cp_full.MessageText[i]))
    markup = ner(mes_cp_full.MessageText[i])
    string = set()
    
    MT = []
    for span in markup.spans:  
        if span.type == 'ORG':
            word = (markup.text[span.start:span.stop]).lower()
            word = m.lemmatize(word)[0]
            if word not in string: # проверка для избежания повторов
                string.add(word)
                ### text boundaries
                for s in sentences:
                    rr = []
                    for r in s: # (rangeL:rangeR), sentence
                        rr.append(r)    
                    if (span.start >= rr[0] and span.stop <= rr[1]):
                        MT.append(rr[2])   
                    
    rows_list = []            
    DP = mes_cp_full.DatePosted[i]
    for s in string:
        c = 0
        new_data = {'MessageText': MT[c], 'DatePosted': DP, 'ticker':s}
        rows_list.append(new_data)
        c+=1
    df = pd.DataFrame(rows_list)
    mes_cp_full = pd.concat([mes_cp_full, df], ignore_index=True)  


100%|██████████| 7538/7538 [03:56<00:00, 31.90it/s]


In [32]:
mes_cp_full

,MessageText,DatePosted,ticker
0,❗️🛢🇷🇺#газ #россия #макро Новак: запасы газа н...,2021-12-29 08:51:59,nvtk
1,ПАО «НОВАТЭК» Тикер NVTK Рекомендация - покупк...,2021-12-23 13:38:22,nvtk
2,🟢 НОВАТЭК договорился о поставках низкоуглерод...,2021-12-23 13:31:19,nvtk
3,☀️ 23.12.2021 📰 Главные темы: • Отбор газа из...,2021-12-23 09:30:01,nvtk
4,🇷🇺🇪🇺#NVTK #водород #аэ #европа НОВАТЭК поста...,2021-12-23 07:47:03,nvtk
...,...,...,...
40832,🔦 Фокус недели #ФН Сегодня 🌎 Вступают в силу...,2021-12-01 08:02:12,макро
40833,🔦 Фокус недели #ФН Сегодня 🌎 Вступают в силу...,2021-12-01 08:02:12,moex
40834,🔦 Фокус недели #ФН Сегодня 🌎 Вступают в силу...,2021-12-01 08:02:12,северсталь
40835,🔦 Фокус недели #ФН Сегодня 🌎 Вступают в силу...,2021-12-01 08:02:12,lkoh


In [33]:
mes_cp_new = mes_cp_full[7538:]
tickers = mes_cp_new['ticker'].tolist()
tickers

['nvtk',
 'новатэк',
 'пао',
 'uniper',
 'sbercib',
 'новатэк',
 'новатэк',
 'uniper',
 'мечесть',
 'новатэк',
 'ъ',
 'казатомпром',
 'газпром',
 'цб',
 'uniper',
 'новатэк',
 'uniper',
 'nvtk',
 'новатэк',
 'sbercib',
 'новатэк',
 'новатэк',
 'nvtk',
 'новатэк',
 'арктик',
 'распоряжение',
 'nvtk',
 'новатэк',
 'роснефть',
 'газпром',
 'новатэк',
 'nvtk',
 'ямал',
 'nvtk',
 'новатэк',
 'новатэк',
 'лукойл',
 'илим',
 'мировой',
 'новатэк',
 'немецкий',
 'rwe',
 'новатэк',
 'немецкий',
 'nvtk',
 'новатэк',
 'новатэк',
 'nvtk',
 'аэрофлот',
 'moex',
 'gltr',
 'русснефть',
 'сд',
 'макро',
 'softline',
 'нмтп',
 'sngs',
 'татнефть',
 'фас',
 'газпром',
 'акрон',
 'тмк',
 'новатэк',
 'globaltrans',
 'цб',
 'smlt',
 'gazp',
 'zhejiang',
 'nvtk',
 'hibiki',
 'новатэк',
 'petrovietnam',
 'saibu',
 'афк',
 'новатэк',
 'опек+',
 'новатэк',
 'яндекс',
 'овк',
 'новатэк',
 'ozon',
 '02',
 'ммвб',
 'sbercib',
 'sber',
 'банк',
 'альфа-банк',
 'райффайзенбанк',
 'sber',
 'газпромбанк',
 'втб',
 'с

In [57]:
!wget https://drive.google.com/file/d/1DNilv-qiNFjCSOhBFZE4JjGbSrAYZ9UP/view?usp=sharing

--2023-11-21 10:12:05--  https://drive.google.com/file/d/1DNilv-qiNFjCSOhBFZE4JjGbSrAYZ9UP/view?usp=sharing
Resolving drive.google.com (drive.google.com)... 74.125.195.101, 74.125.195.113, 74.125.195.102, ...
Connecting to drive.google.com (drive.google.com)|74.125.195.101|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘view?usp=sharing’

view?usp=sharing        [ <=>                ]  82.47K  --.-KB/s    in 0.005s  

2023-11-21 10:12:06 (16.2 MB/s) - ‘view?usp=sharing’ saved [84445]



In [36]:
brands = pd.read_csv('../input/brands-without-rx/all brands - Sheet2.csv') # brands - все issuers(кроме полных названий) с повторами.
brands['abraudurso'] = brands['abraudurso'].str.lower()
brand_list = brands.abraudurso.tolist() 


In [51]:
def sw_info_save(tickers,brand_list):
    stopwords = []
    for ticker in tickers:
        if ticker not in brand_list:
            stopwords.append(ticker)
    print("количество стоп слов:",len(stopwords))
    ssw_set = sorted(set(stopwords))
    print("количество уникальных стоп слов:",len(ssw_set))
    with open("stopwords.txt", 'w') as output:
        output.write(str(list(ssw_set)))
        print("file saved as stopwords.txt")
    return ssw_set

In [52]:
ssw_set = sw_info_save(tickers,brand_list)

количество стоп слов: 24132
количество уникальных стоп слов: 1710
file saved as stopwords.txt


## Уберем RX из issuers и добавим set стоп слов (быстрее чем list)


In [55]:
m = Mystem()
mes_cp_full = mes_012
mes_cp_full = mes_cp_full[['MessageText', 'DatePosted','ticker']]

for i in tqdm(range(mes_cp_full.shape[0])):
    sentences = list(sentenize(mes_cp_full.MessageText[i]))
    markup = ner(mes_cp_full.MessageText[i])
    string = set()
    
    MT = []
    for span in markup.spans:  
        if span.type == 'ORG':
            word = (markup.text[span.start:span.stop]).lower()
            word = m.lemmatize(word)[0]
            if word not in ssw_set:
                if word not in string: # проверка,чтобы избежать повторы
                    ### text boundaries
                    for s in sentences:
                        rr = []
                        for r in s: # (rangeL:rangeR), sentence
                            rr.append(r)    
                        if (span.start >= rr[0] and span.stop <= rr[1]):
                            MT.append(rr[2])
                            string.add(word) # добавляем слово только если для него есть предложение
                #if word in string   
    rows_list = []            
    DP = mes_cp_full.DatePosted[i]
    for c,s in enumerate(string):
        new_data = {'MessageText': MT[c], 'DatePosted': DP, 'ticker':s}
        rows_list.append(new_data)
    df = pd.DataFrame(rows_list)
    mes_cp_full = pd.concat([mes_cp_full, df], ignore_index=True)

100%|██████████| 7538/7538 [03:51<00:00, 32.50it/s]


In [56]:
mes_cp_new = mes_cp_full[7538:]
mes_cp_new

,MessageText,DatePosted,ticker
7538,Фундаментальный фактор Основным позитивным мом...,2021-12-23 13:38:22,новатэк
7539,🟢 НОВАТЭК договорился о поставках низкоуглерод...,2021-12-23 13:31:19,новатэк
7540,• Новатэк (NVTK) заключил соглашение об экспор...,2021-12-23 09:30:01,новатэк
7541,🇷🇺🇪🇺#NVTK #водород #аэ #европа НОВАТЭК поста...,2021-12-23 07:47:03,новатэк
7542,Главный бенефициар сектора - Газпром (GAZP).,2021-12-22 19:29:59,газпром
...,...,...,...
16675,🔦 Фокус недели #ФН Сегодня 🖨 Мосбиржа (MOEX)...,2021-12-02 08:01:34,sber
16676,🔦 Фокус недели #ФН Сегодня 🖨 Мосбиржа (MOEX)...,2021-12-02 08:01:34,северсталь
16677,Компания Самолет (SMLT) провела первый День Ин...,2021-12-01 19:00:00,sber
16678,"Кроме этого, маржинальность ритейлеров может с...",2021-12-01 19:00:00,самолет


### Есть баг: if (span.start >= rr[0] and span.stop <= rr[1]): не всегда выполняется(хотя если слово есть, то для него найдется предложение)
### не понятно,как фиксить - кроме как добавлять слово, только если для него есть предложение

# TODO 
### добавить prev_start, prev_stop, подумать, как дать токену больше чем одно предложение: 1) либо разделение для n, либо вместо MT = [], сделать словарь, MT[word] = concat(sentence, s2), если s2 содержит word(ключ), который уже есть в словаре.

1. ### сделать суперстроки для issuers
### word in string - добавить предложение в MT
